In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_11 import *

## Serializing the model

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=2920)

In [3]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [10]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [8]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [12]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [14]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [15]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [16]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.111322,0.255099,2.042006,0.276000,00:12
1,1.925505,0.350731,1.892934,0.368000,00:10
2,1.800269,0.409186,1.832518,0.380000,00:10
3,1.703318,0.458086,1.853297,0.376000,00:10
4,1.636846,0.495825,1.700234,0.440000,00:10
5,1.593993,0.516139,1.654179,0.468000,00:10
6,1.542583,0.540389,1.618809,0.518000,00:10
7,1.490776,0.565361,1.727768,0.454000,00:10
8,1.455168,0.578609,1.542722,0.534000,00:10
9,1.394935,0.608078,1.662776,0.450000,00:10


In [17]:
st = learn.model.state_dict()

In [19]:
type(st)

collections.OrderedDict

In [20]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [21]:
st['10.bias']

tensor([-0.0334,  0.0152, -0.0176, -0.0377, -0.0186,  0.0017,  0.0038,  0.0584,
         0.0552, -0.0298], device='cuda:0')

In [22]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [23]:
torch.save(st, mdl_path/'iw5')

## Pets

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3127)

In [24]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [25]:
pets.ls()

[PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/annotations'),
 PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/small-96'),
 PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/crappy'),
 PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/small-256'),
 PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/image_gen'),
 PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/models')]

In [26]:
pets_path = pets/'images'

In [27]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [28]:
il

ImageList (7390 items)
[PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/chihuahua_139.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/leonberger_3.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Bombay_127.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Abyssinian_6.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Abyssinian_193.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Bombay_82.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/keeshond_20.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/boxer_140.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/havanese_148.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_12.jpg')...]
Path: /WORKSPACE/.fastai/data/oxford-iiit-pet/images

In [29]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [30]:
random.seed(42)

In [31]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [32]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/chihuahua_139.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Bombay_127.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Abyssinian_6.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Abyssinian_193.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Bombay_82.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/keeshond_20.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/havanese_148.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/german_shorthaired_111.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/saint_bernard_122.jpg'), PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/Sphynx_142.jpg')...]
Path: /WORKSPACE/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/WORKSPACE/.fastai/data/oxford-iiit-pet/images/leonberger_3.jpg'), Posix

In [33]:
n = il.items[0].name; n

'chihuahua_139.jpg'

In [36]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'chihuahua'

In [37]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [38]:
proc = CategoryProcessor()

In [41]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [42]:
', '.join(proc.vocab)

'chihuahua, Bombay, Abyssinian, keeshond, havanese, german_shorthaired, saint_bernard, Sphynx, wheaten_terrier, american_pit_bull_terrier, Egyptian_Mau, Persian, pomeranian, pug, British_Shorthair, beagle, samoyed, yorkshire_terrier, Russian_Blue, miniature_pinscher, japanese_chin, boxer, basset_hound, Siamese, Maine_Coon, newfoundland, staffordshire_bull_terrier, scottish_terrier, Birman, leonberger, Bengal, english_cocker_spaniel, shiba_inu, american_bulldog, great_pyrenees, english_setter, Ragdoll'

In [43]:
ll.valid.x.tfms = val_tfms

In [44]:
c_out = len(proc.vocab)

In [45]:
c_out

37

In [46]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [47]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [48]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.480217,0.084446,3.475090,0.089903,00:09
1,3.286738,0.132893,3.718171,0.071923,00:08
2,3.051330,0.204890,3.307316,0.142462,00:08
3,2.746244,0.291885,2.696892,0.322268,00:08
4,2.433667,0.395980,2.404884,0.399723,00:07


## Custom head

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3265)

In [49]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [50]:
st = torch.load(mdl_path/'iw5')

In [51]:
m = learn.model

In [52]:
m.load_state_dict(st)

In [53]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [54]:
xb,yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [55]:
pred = m_cut(xb)

In [56]:
pred.shape

torch.Size([128, 512, 4, 4])

In [57]:
ni = pred.shape[1]

In [58]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [59]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))

In [60]:
learn.model = m_new

In [61]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.864263,0.294885,2.147944,0.484094,00:08
1,1.942991,0.553922,1.903802,0.557400,00:08
2,1.693834,0.646168,1.977162,0.521438,00:07
3,1.489957,0.718014,1.531473,0.695712,00:07
4,1.292780,0.796160,1.451359,0.735823,00:07


## adapt_model and gradual unfreezing

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3483)

In [62]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children())
               if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [63]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

In [64]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [65]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [66]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.748224,0.309135,2.429798,0.403873,00:06
1,2.390884,0.421629,2.363031,0.406639,00:06
2,2.046957,0.531873,2.042738,0.521438,00:06


In [67]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [68]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.832545,0.605070,1.863176,0.586445,00:07
1,1.718738,0.643918,2.018218,0.528354,00:07
2,1.618642,0.680366,1.768733,0.616874,00:07
3,1.444582,0.743363,1.571919,0.677732,00:07
4,1.269345,0.827209,1.466784,0.731674,00:07


## Batch norm transfer

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3567)

In [69]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [70]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)

def set_grad(m, b):
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [71]:
apply_mod(learn.model, partial(set_grad, b=False))

In [72]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.635844,0.335683,2.338788,0.395574,00:07
1,2.049831,0.522274,1.961583,0.538036,00:07
2,1.803501,0.608220,1.819572,0.600277,00:07


In [73]:
apply_mod(learn.model, partial(set_grad, b=True))

In [74]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.694052,0.654717,1.789426,0.605809,00:07
1,1.641294,0.664467,1.957173,0.558783,00:07
2,1.593316,0.682016,1.784083,0.627939,00:07
3,1.417167,0.755812,1.560836,0.701245,00:07
4,1.245917,0.823759,1.465533,0.724758,00:07


Pytorch already has an `apply` method we can use:

In [75]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3799)

In [76]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [77]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [78]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

In [79]:
a,b = bn_splitter(learn.model)

In [80]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [81]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [82]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [84]:
cb_types

namespace(after_backward='after_backward', after_batch='after_batch', after_cancel_batch='after_cancel_batch', after_cancel_epoch='after_cancel_epoch', after_cancel_train='after_cancel_train', after_epoch='after_epoch', after_fit='after_fit', after_loss='after_loss', after_pred='after_pred', after_step='after_step', begin_batch='begin_batch', begin_epoch='begin_epoch', begin_fit='begin_fit', begin_validate='begin_validate')

In [85]:
cb_types.after_backward

'after_backward'

In [86]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [87]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [88]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [92]:
ParamScheduler??

In [93]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [97]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [101]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.465424,0.388031,2.373250,0.384509,00:07
1,2.189438,0.478626,2.394169,0.401107,00:07
2,1.951736,0.550322,1.888312,0.567082,00:07


In [ ]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [ ]:
learn.fit(5, disc_lr_sched)

## Export

In [ ]:
!./notebook2script.py 11a_transfer_learning.ipynb